In [1]:
from snowflake.snowpark.session import Session
from snowflake.snowpark import functions as F
from snowflake.snowpark.types import *

import pandas as pd

from sklearn import linear_model

import matplotlib.pyplot as plt

%matplotlib inline
import datetime as dt
import numpy as np
import seaborn as sns
import pandas as pd

#Snowflake connection info is saved in config.py
from config import snowflake_conn_prop


# lets import some tranformations functions
from snowflake.snowpark.functions import udf, col, lit, translate, is_null, iff

In [2]:
from snowflake.snowpark import version
print(version.VERSION)
#session.close()
session = Session.builder.configs(snowflake_conn_prop).create()
session.sql("use database {}".format(snowflake_conn_prop['database'])).collect()
session.sql("create schema if not exists {}".format(snowflake_conn_prop['schema'])).collect()
session.sql("use schema {}".format(snowflake_conn_prop['schema'])).collect()
session.sql("use warehouse {}".format(snowflake_conn_prop['warehouse']))
print(session.sql('select current_warehouse(), current_database(), current_schema()').collect())

(0, 11, 0)
[Row(CURRENT_WAREHOUSE()='WH_UWM', CURRENT_DATABASE()='UWM_HOUSEHOLDING', CURRENT_SCHEMA()='HOUSEHOLDING')]


In [3]:
snow = session.sql("select * from BONANZA_TABLA").collect()

In [91]:
df = pd.DataFrame(snow, columns = ['CONTACT_ID','FULL_NAME', 'FIRST_NAME', 'MIDDLE', 'LAST_NAME', 'ADDRESS_LINE_1', 'ADDRESS_LINE_2', 'ADDRESS_LINE_3', 'CITY', 'STATE', 'ZIP', 'HOUSEHOLD_ID','HOUSEHOLD_IND'])
#snowdf.set_index('CONTACT_ID', inplace = True)
df2 = df
df

,CONTACT_ID,FULL_NAME,FIRST_NAME,MIDDLE,LAST_NAME,ADDRESS_LINE_1,ADDRESS_LINE_2,ADDRESS_LINE_3,CITY,STATE,ZIP,HOUSEHOLD_ID,HOUSEHOLD_IND
0,1,Claire Walls,Claire,None,Walls,1031189 Metus St,None,Eugene MD 22207,Eugene,MD,22207,99900001,N
1,2,Derek Kane,Derek,None,Kane,1055973 Semper Av,None,Helena OH 24436,Helena,OH,24436,99900002,N
2,3,Stone Mayer,Stone,None,Mayer,1057190 Consectetuer Street,None,West Valley City MA 45162,West Valley City,MA,45162,99900003,N
3,4,Claudia Riddle,Claudia,None,Riddle,1072885 Semper Street,None,Memphis VA 32674,Memphis,VA,32674,99900004,N
4,5,Fuller Parrish,Fuller,None,Parrish,1076016 Augue Ave,None,Cambridge MN 65252,Cambridge,MN,65252,99900005,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,997,George Hardy,George,None,Hardy,1545 Scelerisque Rd,None,Gresham CO 36762,Gresham,CO,36762,11100020,Y
997,998,Baxter Hardy,Baxter,None,Hardy,1545 Scelerisque Rd,None,Gresham CO 36762,Gresham,CO,36762,11100020,Y
998,999,Randall Hardy,Randall,None,Hardy,1545 Scelerisque Rd,None,Gresham CO 36762,Gresham,CO,36762,11100020,Y
999,1000,Jemima Clayton,Jemima,None,Clayton,1545 Scelerisque Rd,None,Gresham CO 36762,Gresham,CO,36762,11100020,Y


In [120]:
    def encode(s: str):
        if "None" in s:
            s = s.replace("None", " ")
        elif re.search(r'Ave[.]*$', s):
            s = s.replace("Ave", "Avenue")
        elif re.search(r'Av[.]*$', s):
            s = s.replace("Av", "Avenue")
        elif re.search(r'St[.]*$', s):
            s = s.replace("St", "Street")
        elif re.search(r'Rd[.]*$', s):
            s = s.replace("Rd", "Road")
        elif re.search(r'Dr[.]*$', s):
            s = s.replace('Dr', "Drive")
        return s
        #b = s.encode("utf-8")
        #return int.from_bytes(b, byteorder='big')

In [121]:
encode("Sterling Ave.")

'Sterling Avenue.'

In [123]:
df2.apply(lambda row : encode(str(row["ADDRESS_LINE_1"])), axis = 1)
df2.apply(lambda row : encode(str(row["ADDRESS_LINE_2"])), axis = 1)
df2.apply(lambda row : encode(str(row["ADDRESS_LINE_3"])), axis = 1)

0                 Eugene  MD 22207
1                 Helena  OH 24436
2       West Valley City  MA 45162
3                Memphis  VA 32674
4              Cambridge  MN 65252
                   ...            
996              Gresham  CO 36762
997              Gresham  CO 36762
998              Gresham  CO 36762
999              Gresham  CO 36762
1000          Dirty South CA 34581
Length: 1001, dtype: object

In [122]:
df2["UNIQUE IDENTIFIER"]= df2.apply(lambda row : encode(str(row["ADDRESS_LINE_1"])), axis = 1).map(str) + ' ' + df2.apply(lambda row : encode(str(row["ADDRESS_LINE_2"])), axis = 1).map(str) + ' ' + df2.apply(lambda row : encode(str(row["ADDRESS_LINE_3"])), axis = 1).map(str)
df2

,CONTACT_ID,FULL_NAME,FIRST_NAME,MIDDLE,LAST_NAME,ADDRESS_LINE_1,ADDRESS_LINE_2,ADDRESS_LINE_3,CITY,STATE,ZIP,HOUSEHOLD_ID,HOUSEHOLD_IND,ADDRESS,HH_ID,UNIQUE IDENTIFIER
0,1,Claire Walls,Claire,None,Walls,1031189 Metus St,None,Eugene MD 22207,Eugene,MD,22207,99900001,N,1031189 Metus Street Eugene MD 22207,1,1031189 Metus Street Eugene MD 22207
1,2,Derek Kane,Derek,None,Kane,1055973 Semper Av,None,Helena OH 24436,Helena,OH,24436,99900002,N,1055973 Semper Avenue Helena OH 24436,2,1055973 Semper Avenue Helena OH 24436
2,3,Stone Mayer,Stone,None,Mayer,1057190 Consectetuer Street,None,West Valley City MA 45162,West Valley City,MA,45162,99900003,N,1057190 Consectetuer Street West Valley City...,3,1057190 Consectetuer Street West Valley City...
3,4,Claudia Riddle,Claudia,None,Riddle,1072885 Semper Street,None,Memphis VA 32674,Memphis,VA,32674,99900004,N,1072885 Semper Street Memphis VA 32674,4,1072885 Semper Street Memphis VA 32674
4,5,Fuller Parrish,Fuller,None,Parrish,1076016 Augue Ave,None,Cambridge MN 65252,Cambridge,MN,65252,99900005,N,1076016 Augue Avenue Cambridge MN 65252,5,1076016 Augue Avenue Cambridge MN 65252
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,997,George Hardy,George,None,Hardy,1545 Scelerisque Rd,None,Gresham CO 36762,Gresham,CO,36762,11100020,Y,1545 Scelerisque Road Gresham CO 36762,999,1545 Scelerisque Road Gresham CO 36762
997,998,Baxter Hardy,Baxter,None,Hardy,1545 Scelerisque Rd,None,Gresham CO 36762,Gresham,CO,36762,11100020,Y,1545 Scelerisque Road Gresham CO 36762,999,1545 Scelerisque Road Gresham CO 36762
998,999,Randall Hardy,Randall,None,Hardy,1545 Scelerisque Rd,None,Gresham CO 36762,Gresham,CO,36762,11100020,Y,1545 Scelerisque Road Gresham CO 36762,999,1545 Scelerisque Road Gresham CO 36762
999,1000,Jemima Clayton,Jemima,None,Clayton,1545 Scelerisque Rd,None,Gresham CO 36762,Gresham,CO,36762,11100020,Y,1545 Scelerisque Road Gresham CO 36762,999,1545 Scelerisque Road Gresham CO 36762


In [110]:
df2['HH_ID'] = df2.groupby('UNIQUE IDENTIFIER')['CONTACT_ID'].transform('max')

In [112]:
r = "SELECT * FROM df where HOUSEHOLD_ID = 11100003"
r1 = "SELECT * FROM df2 where HH_ID = 550"
sqldf(r1, globals())

,CONTACT_ID,FULL_NAME,FIRST_NAME,MIDDLE,LAST_NAME,ADDRESS_LINE_1,ADDRESS_LINE_2,ADDRESS_LINE_3,CITY,STATE,ZIP,HOUSEHOLD_ID,HOUSEHOLD_IND,ADDRESS,HH_ID,UNIQUE IDENTIFIER
0,546,Octavius Cortez,Octavius,None,Cortez,5229 Vitae St,Apt 182,Fort Worth AL 58712,Fort Worth,AL,58712,11100009,Y,5229 Vitae Street Apt 182 Fort Worth AL 58712,550,5229 Vitae Street Apt 182 Fort Worth AL 58712
1,547,Sean Gardner,Sean,None,Gardner,5229 Vitae St,Apt 182,Fort Worth AL 58712,Fort Worth,AL,58712,11100009,Y,5229 Vitae Street Apt 182 Fort Worth AL 58712,550,5229 Vitae Street Apt 182 Fort Worth AL 58712
2,548,Scarlett Lee Mcgowan,Scarlett,Lee,Mcgowan,5229 Vitae St,Apt 182,Fort Worth AL 58712,Fort Worth,AL,58712,11100009,Y,5229 Vitae Street Apt 182 Fort Worth AL 58712,550,5229 Vitae Street Apt 182 Fort Worth AL 58712
3,549,Stephanie John Mckinney,Stephanie,John,Mckinney,5229 Vitae St,Apt 182,Fort Worth AL 58712,Fort Worth,AL,58712,11100009,Y,5229 Vitae Street Apt 182 Fort Worth AL 58712,550,5229 Vitae Street Apt 182 Fort Worth AL 58712
4,550,Carly Baldwin,Carly,None,Baldwin,5229 Vitae St,Apt 182,Fort Worth AL 58712,Fort Worth,AL,58712,11100009,Y,5229 Vitae Street Apt 182 Fort Worth AL 58712,550,5229 Vitae Street Apt 182 Fort Worth AL 58712


In [113]:
from pandasql import sqldf
q = "SELECT HOUSEHOLD_ID, COUNT(HH_ID) AS NO_OF_MEMBERS FROM df2 group by HH_ID having NO_OF_MEMBERS!= 1 order by HOUSEHOLD_ID"
q1 = "SELECT HOUSEHOLD_ID, COUNT(HOUSEHOLD_ID) AS NO_OF_MEMBERS FROM df group by HOUSEHOLD_ID having NO_OF_MEMBERS!= 1"
q2 = "SELECT * FROM df2 WHERE HOUSEHOLD_ID < 11100021"
p = sqldf(q, globals())
p1 = sqldf(q1, globals())
p2 = sqldf(q2, globals())
p2

,CONTACT_ID,FULL_NAME,FIRST_NAME,MIDDLE,LAST_NAME,ADDRESS_LINE_1,ADDRESS_LINE_2,ADDRESS_LINE_3,CITY,STATE,ZIP,HOUSEHOLD_ID,HOUSEHOLD_IND,ADDRESS,HH_ID,UNIQUE IDENTIFIER
0,11,Clementine J Gaines,Clementine,J,Gaines,1190 Porta Street,None,Jacksonville MN 73876,Jacksonville,MN,73876,11100001,Y,1190 Porta Street Jacksonville MN 73876,12,1190 Porta Street Jacksonville MN 73876
1,12,Althea Gaines,Althea,None,Gaines,1190 Porta Street,None,Jacksonville MN 73876,Jacksonville,MN,73876,11100001,Y,1190 Porta Street Jacksonville MN 73876,12,1190 Porta Street Jacksonville MN 73876
2,26,Burton Ramirez,Burton,None,Ramirez,1539541 Aenean Rd,None,Vancouver HI 26858,Vancouver,HI,26858,11100002,Y,1539541 Aenean Road Vancouver HI 26858,27,1539541 Aenean Road Vancouver HI 26858
3,27,Gemma Ramirez,Gemma,None,Ramirez,1539541 Aenean Rd,None,Vancouver HI 26858,Vancouver,HI,26858,11100002,Y,1539541 Aenean Road Vancouver HI 26858,27,1539541 Aenean Road Vancouver HI 26858
4,67,Donovan Silva,Donovan,None,Silva,2433648 Ante St,None,Grand Island CO 47088,Grand Island,CO,47088,11100003,Y,2433648 Ante Street Grand Island CO 47088,71,2433648 Ante Street Grand Island CO 47088
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,970,Kaseem Lancaster,Kaseem,None,Lancaster,7764 Arcu St,None,Iowa City AK 99663,Iowa City,AK,99663,11100019,Y,7764 Arcu Street Iowa City AK 99663,970,7764 Arcu Street Iowa City AK 99663
64,997,George Hardy,George,None,Hardy,1545 Scelerisque Rd,None,Gresham CO 36762,Gresham,CO,36762,11100020,Y,1545 Scelerisque Road Gresham CO 36762,999,1545 Scelerisque Road Gresham CO 36762
65,998,Baxter Hardy,Baxter,None,Hardy,1545 Scelerisque Rd,None,Gresham CO 36762,Gresham,CO,36762,11100020,Y,1545 Scelerisque Road Gresham CO 36762,999,1545 Scelerisque Road Gresham CO 36762
66,999,Randall Hardy,Randall,None,Hardy,1545 Scelerisque Rd,None,Gresham CO 36762,Gresham,CO,36762,11100020,Y,1545 Scelerisque Road Gresham CO 36762,999,1545 Scelerisque Road Gresham CO 36762


In [114]:
#for each in p["HH_ID"]:
#   print(each)
p1

,HOUSEHOLD_ID,NO_OF_MEMBERS
0,11100001,2
1,11100002,2
2,11100003,5
3,11100004,4
4,11100005,3
5,11100006,3
6,11100007,2
7,11100008,7
8,11100009,5
9,11100010,3


In [115]:
p

,HOUSEHOLD_ID,NO_OF_MEMBERS
0,11100001,2
1,11100002,2
2,11100003,5
3,11100004,4
4,11100005,3
5,11100006,3
6,11100007,2
7,11100008,2
8,11100008,5
9,11100009,5


In [116]:
pd.concat([p,p1]).drop_duplicates(keep=False)

,HOUSEHOLD_ID,NO_OF_MEMBERS
7,11100008,2
8,11100008,5
21,99900667,2
22,99900669,2
7,11100008,7


In [119]:
sqldf("select * from df where HOUSEHOLD_ID = 11100008", globals())

,CONTACT_ID,FULL_NAME,FIRST_NAME,MIDDLE,LAST_NAME,ADDRESS_LINE_1,ADDRESS_LINE_2,ADDRESS_LINE_3,CITY,STATE,ZIP,HOUSEHOLD_ID,HOUSEHOLD_IND,ADDRESS,HH_ID,UNIQUE IDENTIFIER
0,431,Ursula Parrish,Ursula,None,Parrish,8392414 Ridiculus Avenue,None,Broken Arrow NE 62835,Broken Arrow,NE,62835,11100008,Y,8392414 Ridiculus Avenue Broken Arrow NE 62835,432,8392414 Ridiculus Avenue Broken Arrow NE 62835
1,432,Rosalyn Parrish,Rosalyn,None,Parrish,8392414 Ridiculus Avenue,None,Broken Arrow NE 62835,Broken Arrow,NE,62835,11100008,Y,8392414 Ridiculus Avenue Broken Arrow NE 62835,432,8392414 Ridiculus Avenue Broken Arrow NE 62835
2,478,Aretha Gamble,Aretha,None,Gamble,9371491 Lorem Ave,None,Cheyenne IA 67171,Cheyenne,IA,67171,11100008,Y,9371491 Lorem Avenue Cheyenne IA 67171,482,9371491 Lorem Avenue Cheyenne IA 67171
3,479,Derek Gamble,Derek,None,Gamble,9371491 Lorem Ave,None,Cheyenne IA 67171,Cheyenne,IA,67171,11100008,Y,9371491 Lorem Avenue Cheyenne IA 67171,482,9371491 Lorem Avenue Cheyenne IA 67171
4,480,Shad Robbins,Shad,None,Robbins,9371491 Lorem Ave,None,Cheyenne IA 67171,Cheyenne,IA,67171,11100008,Y,9371491 Lorem Avenue Cheyenne IA 67171,482,9371491 Lorem Avenue Cheyenne IA 67171
5,481,Odysseus Dodson,Odysseus,None,Dodson,9371491 Lorem Ave,None,Cheyenne IA 67171,Cheyenne,IA,67171,11100008,Y,9371491 Lorem Avenue Cheyenne IA 67171,482,9371491 Lorem Avenue Cheyenne IA 67171
6,482,Cora Gamble,Cora,None,Gamble,9371491 Lorem Ave,None,Cheyenne IA 67171,Cheyenne,IA,67171,11100008,Y,9371491 Lorem Avenue Cheyenne IA 67171,482,9371491 Lorem Avenue Cheyenne IA 67171


In [69]:
lr = make_pipeline(LinearRegression())
rnc = RadiusNeighborsClassifier(radius=0, outlier_label = "most frequent")
model = make_pipeline(rnc)
model.fit(X_train,y_train.values.ravel())

Pipeline(steps=[('radiusneighborsclassifier',
                 RadiusNeighborsClassifier(outlier_label='most frequent',
                                           radius=0))])

In [124]:
session.close()